### EY Datascience Challenge ###
**Import relevant libraries and datasets**

divide the test data into dataset for prediction and dataset for evaluation

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
#Reading training data
df = pd.read_csv ("data_train.csv")
df = df.loc[:,'hash':'y_exit']
df.fillna('',inplace=True)

In [3]:
#Reading test data
df_test = pd.read_csv ("data_test.csv")
df_test = df_test.loc[:,'hash':'y_exit']
df_test.fillna('',inplace=True)

In [4]:
#divide the test data into pred and eval
df_test['time_entry']=pd.to_timedelta(df_test['time_entry'])
df_test['time_exit']=pd.to_timedelta(df_test['time_exit'])

df_test_pred = df_test[df_test['x_exit'] == ""]
df_test_eval = df_test[df_test['x_exit'] != ""]

**First Look at the Data**

Type and informations of the datasets

In [6]:
print (df.info())
print (df_test_eval.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 11 columns):
hash             814262 non-null object
trajectory_id    814262 non-null object
time_entry       814262 non-null object
time_exit        814262 non-null object
vmax             814262 non-null object
vmin             814262 non-null object
vmean            814262 non-null object
x_entry          814262 non-null float64
y_entry          814262 non-null float64
x_exit           814262 non-null float64
y_exit           814262 non-null float64
dtypes: float64(4), object(7)
memory usage: 68.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 169422 entries, 0 to 202935
Data columns (total 11 columns):
hash             169422 non-null object
trajectory_id    169422 non-null object
time_entry       169422 non-null timedelta64[ns]
time_exit        169422 non-null timedelta64[ns]
vmax             169422 non-null object
vmin             169422 non-null object
vmean            1

In [7]:
#Some statistics
df.describe()

,x_entry,y_entry,x_exit,y_exit
count,8.142620e+05,8.142620e+05,8.142620e+05,8.142620e+05
mean,3.760411e+06,-1.922153e+07,3.760426e+06,-1.922181e+07
std,8.974393e+03,7.632234e+04,8.904451e+03,7.515560e+04
min,3.741027e+06,-1.938291e+07,3.740998e+06,-1.937688e+07
25%,3.755174e+06,-1.927454e+07,3.755359e+06,-1.927308e+07
50%,3.760071e+06,-1.923020e+07,3.760076e+06,-1.923031e+07
75%,3.767509e+06,-1.916999e+07,3.767455e+06,-1.917193e+07
max,3.777099e+06,-1.904266e+07,3.777055e+06,-1.904666e+07


In [8]:
df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,,,,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,,,,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,,,,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,,,,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,,,,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07


In [12]:
#Prepare time stayed in the city hall (seconds)
df['time_entry_seconds'] = pd.to_timedelta(df['time_entry']).dt.total_seconds()
df['time_exit_seconds']=pd.to_timedelta(df['time_exit']).dt.total_seconds()
df['time_stayed_seconds']=df['time_exit_seconds']-df['time_entry_seconds']
df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,time_entry_seconds,time_exit_seconds,time_stayed_seconds
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,,,,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,25471.0,25712.0,241.0
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,,,,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,26434.0,26742.0,308.0
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,,,,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,28412.0,29005.0,593.0
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,,,,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,29870.0,31043.0,1173.0
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,,,,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,52689.0,52689.0,0.0


In [16]:
#prepare whether entry point is in cityhall
df_target = df.loc[:,['x_entry','y_entry']]

target_x1 = df_target['x_entry']>=3750901.5068
target_x2 = df_target['x_entry']<=3770901.5068

target_x = target_x1 & target_x2

target_y1 = df_target['y_entry'] >= -19268905.6133
target_y2 = df_target['y_entry'] <= -19208905.6133

target_y = target_y1 & target_y2

df['entry_inside'] = 1*(target_x & target_y)

df.head()
df.entry_inside.describe()

count    814262.000000
mean          0.291157
std           0.454296
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: entry_inside, dtype: float64

In [17]:
#Prepare train data
df_train=df.loc[:,['time_stayed_seconds','entry_inside']]

train_data = df_train.values

train_data

array([[241.,   0.],
       [308.,   0.],
       [593.,   0.],
       ...,
       [  0.,   0.],
       [409.,   0.],
       [439.,   0.]])

In [18]:
#prepare training label
df_target = df.loc[:,['x_exit','y_exit']]

target_x1 = df_target['x_exit']>=3750901.5068
target_x2 = df_target['x_exit']<=3770901.5068

target_x = target_x1 & target_x2

target_y1 = df_target['y_exit'] >= -19268905.6133
target_y2 = df_target['y_exit'] <= -19208905.6133

target_y = target_y1 & target_y2

train_label = 1*(target_x & target_y)

print (train_label.describe())

train_label = train_label.values

train_label = to_categorical(train_label)

count    814262.000000
mean          0.298690
std           0.457684
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
dtype: float64


In [19]:
#Build model
model = Sequential()
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_label, epochs=1)

Epoch 1/1
814262/814262 [==============================] - 73s 90us/step - loss: 1.1854 - acc: 0.9038


In [21]:
#Preparing predicting data
df_test_pred['time_entry_seconds'] = pd.to_timedelta(df_test_pred['time_entry']).dt.total_seconds()
df_test_pred['time_exit_seconds']=pd.to_timedelta(df_test_pred['time_exit']).dt.total_seconds()
df_test_pred['time_stayed_seconds']=df_test_pred['time_exit_seconds']-df_test_pred['time_entry_seconds']


df_test_pred.head()

/Users/minseok/anaconda3/envs/py3.5_tf1.1/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/minseok/anaconda3/envs/py3.5_tf1.1/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/minseok/anaconda3/envs/py3.5_tf1.1/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,time_entry_seconds,time_exit_seconds,time_stayed_seconds
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,15:03:32,15:10:32,,,,3.773118e+06,-1.914490e+07,,,54212.0,54632.0,420.0
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,15:29:09,15:30:56,,,,3.769978e+06,-1.934136e+07,,,55749.0,55856.0,107.0
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,15:26:08,15:26:08,,,,3.757468e+06,-1.923860e+07,,,55568.0,55568.0,0.0
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,15:35:18,15:42:05,,,,3.760505e+06,-1.935500e+07,,,56118.0,56525.0,407.0
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,14:54:07,15:05:14,,,,3.766319e+06,-1.917013e+07,,,53647.0,54314.0,667.0


In [22]:
#prepare predicting data
df_target = df_test_pred.loc[:,['x_entry','y_entry']]

target_x1 = df_target['x_entry']>=3750901.5068
target_x2 = df_target['x_entry']<=3770901.5068

target_x = target_x1 & target_x2

target_y1 = df_target['y_entry'] >= -19268905.6133
target_y2 = df_target['y_entry'] <= -19208905.6133

target_y = target_y1 & target_y2

df_test_pred['entry_inside'] = 1*(target_x & target_y)

df_test_pred.head()
df_test_pred.entry_inside.describe()

/Users/minseok/anaconda3/envs/py3.5_tf1.1/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    33515.000000
mean         0.284768
std          0.451311
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: entry_inside, dtype: float64

In [23]:
#prepare test_data_pred
test_data_pred = df_test_pred.loc[:,['time_stayed_seconds','entry_inside']]
test_data_pred = test_data_pred.values

test_data_pred

array([[420.,   0.],
       [107.,   0.],
       [  0.,   1.],
       ...,
       [  0.,   0.],
       [331.,   0.],
       [  0.,   0.]])

In [27]:
#Make predictions
predictions = model.predict(test_data_pred)

predictions = (predictions[:,1]>0.5)*1

print (predictions)



[0 0 1 ... 0 0 0]


In [28]:
#Save
id = pd.DataFrame(df_test_pred['trajectory_id'])

target = pd.DataFrame(predictions)
target.columns = ['target']


output = pd.concat([id.reset_index(drop=True),target.reset_index(drop=True)], axis=1)
output.columns = ['id', 'target']
output.to_csv("try_2.csv", index=False)

In [32]:
#debugging
print("test_data_pred", test_data_pred.shape)
print("predictions", predictions.shape)
print("df_test_pred", df_test_pred.shape)
print("id", id.shape)
print("target", target.shape)
print("output", output.shape)

print (target.describe())

print(output)

test_data_pred (33515, 2)
predictions (33515,)
df_test_pred (33515, 15)
id (33515, 1)
target (33515, 1)
output (33515, 2)
             target
count  33515.000000
mean       0.258571
std        0.437856
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
                                                id  target
0       traj_00032f51796fd5437b238e3a9823d13d_31_5       0
1      traj_000479418b5561ab694a2870cc04fd43_25_10       0
2       traj_000506a39775e5bca661ac80e3f466eb_29_5       1
3        traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4       0
4        traj_00063a4f6c12e1e4de7d876580620667_3_4       0
5       traj_0006535be25bb52dd06983447880c964_5_12       0
6       traj_0006f84bb33ec929d1cda7686f861d0a_31_3       1
7      traj_00093ae562586aed0e053b8431e8ace4_23_10       0
8       traj_000c739e444a70e1804d757a0580caaa_31_3       1
9       traj_000d479078af08618bddc7f09082b8c3_11_6       0
10     traj_000d6fba12a7d06defe90e3c9162c11e_29_1